In [6]:
from tqdm.notebook import tqdm
import numpy as np
np.set_printoptions(precision=3, suppress=True, linewidth=120);

from causalbenchmark import create_graph, util
from causalbenchmark.graphs.builders import DifficultyBuilder, MechanismCorrelationBuilder

In [2]:
builder = DifficultyBuilder()

In [22]:
spec = {'V1': [0.4, 0.6], 'X': {'V1': -0.5}, 'Y': {'X': 0.2, 'V1': 0.25}}

In [23]:
scm = builder.generate_scm('simpson', spec)
scm

SimpsonsParadox(V1, X(V1), Y(V1, X))

In [24]:
scm.Y.parents

('V1', 'X')

In [25]:
scm.X.param

array([0.838, 0.045])

In [26]:
scm.Y.param

array([[0.154, 0.387],
       [0.533, 0.789]])

In [27]:
scm.aggregate_stats()

(0.4674862740502898, 0.40344662650628965)

In [110]:
spec = {'V1': [0.4, 0.6], 'X': {'V1': -0.6}, 'Y': {'X': 0.2, 'V1': 0.4}}

In [213]:
bad = 0
N = 1000
for _ in tqdm(range(N)):
	control, treatment = builder.generate_scm('simpson', spec).aggregate_stats()
	if treatment > control:
		bad += 1
print(f'bad: {bad/N:.2%}')
bad

  0%|          | 0/1000 [00:00<?, ?it/s]

bad: 100.00%


1000

In [7]:
cb = MechanismCorrelationBuilder()

In [214]:
# confirmed: treatment is good
spec = {'V1': [0.4, 0.6], 'X': {'V1': -0.7}, 'Y': {'X': 0.05, 'V1': 0.6}}
spec = {'V1': [0.4, 0.6], 'X': {'V1': -0.5}, 'Y': {'X': 0.2, 'V1': 0.7}}
# spec = {'V1': [0.4, 0.6], 'X': {'V1': -0.8}, 'Y': {'X': 0.3, 'V1': 0.55}}
# spec = {'V1': [0.4, 0.6], 'X': {'V1': -0.7}, 'Y': {'X': 0.2, 'V1': 0.6}}
# spec = {'V1': [0.4, 0.6], 'X': {'V1': -0.8}, 'Y': {'X': 0.1, 'V1': 0.55}}
# spec = {'V1': [0.4, 0.6], 'X': {'V1': -0.4}, 'Y': {'X': 0.1, 'V1': 0.71}}
# spec = {'V1': [0.4, 0.6], 'X': {'V1': -0.8}, 'Y': {'X': 0.2, 'V1': 0.55}}
#
# # reverse: treatment is bad
# spec = {'V1': [0.4, 0.6], 'X': {'V1': -0.8}, 'Y': {'X': -0.1, 'V1': -0.55}}
# spec = {'V1': [0.4, 0.6], 'X': {'V1': -0.7}, 'Y': {'X': -0.05, 'V1': -0.6}}
# spec = {'V1': [0.4, 0.6], 'X': {'V1': -0.8}, 'Y': {'X': -0.1, 'V1': -0.55}}
# spec = {'V1': [0.4, 0.6], 'X': {'V1': -0.5}, 'Y': {'X': -0.2, 'V1': -0.7}}
# spec = {'V1': [0.4, 0.6], 'X': {'V1': -0.4}, 'Y': {'X': -0.1, 'V1': -0.71}}
# spec = {'V1': [0.4, 0.6], 'X': {'V1': -0.8}, 'Y': {'X': -0.3, 'V1': -0.55}}

In [215]:
model = cb.generate_scm('simpson', spec)
# model

In [216]:
def objective(params):
	return model.subject.set_parameters(params).marginals(X=1)['Y'] - model.subject.set_parameters(params).marginals(X=0)['Y']

In [217]:
model.find_bounds(objective)

[-0.564399999999996, -0.06999999999999945]

In [221]:
scm = builder.generate_scm('simpson', spec)
print(scm.Y.param)
scm.aggregate_stats()

[[0.004 0.221]
 [0.727 0.928]]


(0.5669248426659519, 0.3340683112156557)

In [97]:
# objective(scm.get_parameters())

In [23]:
x0 = None
self = model
from scipy import optimize as opt

In [24]:
if x0 is None:
	x0 = self._initial_parameters()

sol = opt.minimize(objective, x0, constraints=self.constraints, method=self.method)
nsol = opt.minimize(lambda x: -objective(x), x0, constraints=self.constraints, method=self.method)

if not sol.success or not nsol.success:
	print('WARNING: optimization failed')
	print(sol)
	print(nsol)

lb = sol.fun
if isinstance(lb, np.ndarray):
	lb = lb.item()

ub = -nsol.fun
if isinstance(ub, np.ndarray):
	ub = ub.item()


In [25]:
nsol.x

array([0.5 , 0.9 , 0.1 , 0.01, 0.39, 0.61, 0.99])

In [26]:
self.subject.set_parameters(nsol.x).aggregate_stats()

(0.5499999999999998, 0.45000000000000023)